# Transformer的PyTorch实现

> 本文由**罗周杨 stupidme.me.lzy@gmail.com**原创，转载请注明原作者和出处。
> 本文严禁用于商业用途。

之前研究了一番**Transformer**模型，也有一个笔记[Transformer学习笔记](transformer.ipynb)，现在我们用PyTorch实现。


## Position encoding的实现
再回顾一下什么是Position encoding。我们知道Transformer模型由于没有使用RNN，必须使用额外的手段来获取文本序列的顺序（或者说位置）信息。我们的Position encoding(PE)就是实现这个目标而出现的。

Position encoding和Word embedding十分相似。Word embeddings是对词语的内容进行嵌入，而Position encoding是对词语的位置进行嵌入。所以，Position encoding还有个很流行的名字，叫做Position embedding。

下面我们看代码吧，然后讲解代码，你就可以弄懂其中的所有细节了。

In [ ]:
import torch
import torch.nn as nn


class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, max_seq_len):
        """初始化。
        
        Args:
            d_model: 一个标量。模型的维度，论文默认是512
            max_seq_len: 一个标量。文本序列的最大长度
        """
        super(PositionalEncoding, self).__init__()
        
        # 根据论文给的公式，构造出PE矩阵
        position_encoding = np.array([
          [pos / np.pow(10000, 2.0 * (j // 2) / d_model) for j in range(d_model)]
          for pos in range(max_seq_len)])
        # 偶数列使用sin，奇数列使用cos
        position_encoding[:, 0::2] = np.sin(position_encoding[:, 0::2])
        position_encoding[:, 1::2] = np.cos(position_encoding[:, 1::2])

        # 在PE矩阵的第一行，加上一行全是0的向量，代表这`PAD`的positional encoding
        # 在word embedding中也经常会加上`UNK`，代表位置单词的word embedding，两者十分类似
        # 那么为什么需要这个额外的PAD的编码呢？很简单，因为文本序列的长度不一，我们需要对齐，
        # 短的序列我们使用0在结尾补全，我们也需要这些补全位置的编码，也就是`PAD`对应的位置编码
        pad_row = torch.zeros([1, d_model])
        position_encoding = torch.cat((pad_row, position_encoding))
        
        # 嵌入操作，+1是因为增加了`PAD`这个补全位置的编码，
        # Word embedding中如果词典增加`UNK`，我们也需要+1。看吧，两者十分相似
        self.position_encoding = nn.Embedding(max_seq_len + 1, d_model)
        self.position_encoding.weight = nn.Parameter(position_encoding,
                                                     requires_grad=False)
    def forward(self, input_len):
        """神经网络的前向传播。

        Args:
          input_len: 一个张量，形状为[BATCH_SIZE, 1]。每一个张量的值代表这一批文本序列中对应的长度。

        Returns:
          返回这一批序列的位置编码，进行了对齐。
        """
        
        # 找出这一批序列的最大长度
        max_len = torch.max(input_len)
        tensor = torch.cuda.LongTensor if input_len.is_cuda else torch.LongTensor
        # 对每一个序列的位置进行对齐，在原序列位置的后面补上0
        # 这里range从1开始也是因为要避开PAD(0)的位置
        input_pos = tensor(
          [list(range(1, len + 1)) + [0] * (max_len - len) for len in input_len])
        return self.position_encoding(input_pos)
    

## Layer normalization的实现
Layer nomalization和Batch normalization的比较，在文章开头提到的笔记已经有说明了。

接下来就来实现它吧。

In [ ]:
import torch
import torch.nn as nn


class LayerNorm(nn.Module):
    """实现LayerNorm。其实PyTorch已经实现啦，见nn.LayerNorm。"""

    def __init__(self, features, epsilon=1e-6):
    """Init.

    Args:
        features: 就是模型的维度。论文默认512
        epsilon: 一个很小的数，防止数值计算的除0错误
    """
    super(LayerNorm, self).__init__()
    # weights
    self.gamma = nn.Parameter(torch.ones(features))
    # bias
    self.beta = nn.Parameter(torch.zeros(features))
    self.epsilon = epsilon

    def forward(self, x):
    """Forward pass.

    Args:
        x: Input tensor, with shape of [B, L, D]
    """
    # 根据公式进行归一化
    # 在X的最后一个维度求均值，最后一个维度就是模型的维度
    mean = x.mean(-1, keepdim=True)
    # 在X的最后一个维度求方差，最后一个维度就是模型的维度
    std = x.std(-1, keepdim=True)
    return self.gamma * (x - mean) / (std + self.epsilon) + self.beta


## Scaled dot-product attention的实现

具体的原理在另一个笔记有提到。咱们现在实现它，这个代码也比较简单，根据论文的公式即可。唯一要注意的是，有个mask。

代码如下：

In [ ]:
import torch
import torch.nn as nn


class ScaledDotProductAttention(nn.Module):

    def __init__(self, attention_dropout=0.0, scale=True):
        super(ScaledDotProductAttention, self).__init__()
        self.scale = scale
        self.dropout = nn.Dropout(attention_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, mask=None):
    """Forward pass.

    Args:
      q: Queries tensor, with shape of [B, L_q, D_q]
      k: Keys tensor, with shape of [B, L_k, D_k]
      v: Values tensor, with shape of [B, L_v, D_v]
      mask: A ByteTensor, binary mask. If not None, do mask.
    """
        attention = torch.bmm(q, k.transpose(1, 2))
        if self.scale:
            d_k = k.size(-1)  # get model's dimension or num_units
            attention = attention / np.sqrt(d_k)
        if mask:
            attention = attention.masked_fill_(mask, -np.inf)
        attention = self.softmax(attention)
        attention = self.dropout(attention)
        context = torch.bmm(attention, v)
        return context, attention